In [ ]:
import pandas as pd
import os
from IPython.display import display
from matplotlib import pyplot as plt
import plotly.express as px
import seaborn as sns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
df2024 = pd.read_csv(os.path.join(os.path.expanduser("~"), "work", "Downloads", "finances", "chase", "Chase1567_Activity2024.csv"))
df2025 = pd.read_csv(os.path.join(os.path.expanduser("~"), "work", "Downloads", "finances", "chase", "Chase1567_Activity2025.csv"))

In [ ]:
set(df2025.Category.unique()) - set(df2024.Category.unique())

In [ ]:
set(df2024.Category.unique()) - set(df2025.Category.unique())

In [ ]:
df2025.Category.unique()

In [ ]:
# 2024
df2024["Transaction Date"] = pd.to_datetime(df2024["Transaction Date"])

uber_df_2024 = df2024[
    (df2024["Type"] == "Sale") & 
    (df2024["Category"] == "Travel") & 
    (df2024["Description"].str.contains("UBER", case=False, na=False))
].copy()

# 2025
df2025["Transaction Date"] = pd.to_datetime(df2025["Transaction Date"])

uber_df_2025 = df2025[
    (df2025["Type"] == "Sale") & 
    (df2025["Category"] == "Travel") & 
    (df2025["Description"].str.contains("UBER|YANGO|CARDPAY", case=False, na=False))
].copy()

### Uber & Yango spending

In [ ]:
uber_df_2025["Month"] = uber_df_2025["Transaction Date"].dt.to_period("M")
monthly_spending = uber_df_2025.groupby("Month")["Amount"].sum()
average_spending = monthly_spending.mean()

plt.figure(figsize=(10, 5))
monthly_spending.plot(kind="bar", color="#F5A623", edgecolor="black")
plt.axhline(y=average_spending, color="#117A65", linestyle="dashed", linewidth=2, label=f"Avg: ${average_spending:.2f}")
plt.legend()
plt.title("Monthly Uber Spending - 2025")
plt.xlabel("Month")
plt.ylabel("Total Spending ($)")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

In [ ]:
uber_df_2024["Month"] = uber_df_2024["Transaction Date"].dt.to_period("M")
monthly_spending = uber_df_2024.groupby("Month")["Amount"].sum()
average_spending = monthly_spending.mean()

plt.figure(figsize=(10, 5))
monthly_spending.plot(kind="bar", color="royalblue", edgecolor="black")
plt.axhline(y=average_spending, color="#E94E77", linestyle="dashed", linewidth=2, label=f"Avg: ${average_spending:.2f}")
plt.legend()
plt.title("Monthly Uber Spending - 2024")
plt.xlabel("Month")
plt.ylabel("Total Spending ($)")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

## Credit card spending

In [ ]:
categories = df2025['Category'].unique()
category_colors = sns.color_palette("tab20", n_colors=len(categories))
category_color_map = dict(zip(categories, category_colors))

In [ ]:
df2025["Transaction Date"] = pd.to_datetime(df2025["Transaction Date"], errors="coerce")
df2025["Month"] = df2025["Transaction Date"].dt.to_period("M")
df2025_spending = df2025[df2025["Type"] == "Sale"].copy()
df2025_spending["Category"] = df2025_spending["Category"].fillna("Other")
df2025_spending["Amount"] = df2025_spending["Amount"].apply(lambda x: abs(x))
monthly_category_spending = (
    df2025_spending.groupby(["Month", "Category"])["Amount"]
    .sum()
    .unstack(fill_value=0)
)
# Reindex to ensure all expected categories appear in consistent order
monthly_category_spending = monthly_category_spending.reindex(columns=categories, fill_value=0)

# Compute total monthly spending and average
monthly_totals = monthly_category_spending.sum(axis=1)
average_spending = monthly_totals.mean()

plt.figure(figsize=(12, 6))
monthly_category_spending.plot(
    kind="bar",
    stacked=True,
    figsize=(12, 6),
    edgecolor="black",
    color=[category_color_map[cat] for cat in monthly_category_spending.columns]
)
for i, value in enumerate(monthly_totals):
    plt.text(i, value + 1, f"${value:.2f}", ha="center", va="bottom", fontsize=9)
    
# Add average line
plt.axhline(y=average_spending, color="#117A65", linestyle="dashed", linewidth=2, label=f"Avg: ${average_spending:.2f}")

plt.title("Credit Card Monthly Spending by Category - 2025")
plt.xlabel("Month")
plt.ylabel("Total Spending ($)")
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Move legend outside plot
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# PIE CHART 2025
# spending by category
spending = df2025[df2025["Type"] == 'Sale'].copy()
spending["Amount"] = spending["Amount"].abs()
total_spent = spending["Amount"].sum()

# Group by category (assuming the column is named 'category')
category_spending = spending.groupby("Category")["Amount"].sum().sort_values(ascending=False).reset_index()

fig = px.pie(
    category_spending,
    values="Amount",
    names="Category",
    title=f"Spending Distribution by Category - 2025 (Total: ${total_spent:,.2f})",
    color_discrete_sequence=px.colors.sequential.Blues,
    hole=0.3  # For a donut chart look; remove this line if you want a regular pie
)

fig.update_traces(
    textposition="auto",  # Automatically choose inside or outside
    textinfo="label+percent",
    textfont_size=12,
    insidetextorientation="radial",
    pull=[0.03 if v < 0.05 * total_spent else 0 for v in category_spending["Amount"]],  # Slightly pull out small slices
    showlegend=True
)

# Force small slices' labels to go outside if they won't fit
fig.update_layout(
    uniformtext_minsize=12,
    uniformtext_mode="hide",  # Hide if text can't fit
    margin=dict(t=60, b=40, l=40, r=40),
    showlegend=False,
)


fig.show()

In [ ]:
df2024["Transaction Date"] = pd.to_datetime(df2024["Transaction Date"], errors="coerce")
df2024["Month"] = df2024["Transaction Date"].dt.to_period("M")
df2024_spending = df2024[df2024["Type"] == "Sale"].copy()
df2024_spending["Amount"] = df2024_spending["Amount"].apply(lambda x: abs(x))
df2024_spending["Category"] = df2024_spending["Category"].fillna("Other")
monthly_category_spending = (
    df2024_spending.groupby(["Month", "Category"])["Amount"]
    .sum()
    .unstack(fill_value=0)
)

monthly_category_spending = monthly_category_spending.reindex(columns=categories, fill_value=0)

# Compute total monthly spending and average
monthly_totals = monthly_category_spending.sum(axis=1)
average_spending = monthly_totals.mean()
total_spent = df2024_spending["Amount"].abs().sum()

plt.figure(figsize=(12, 6))
monthly_category_spending.plot(
    kind="bar",
    stacked=True,
    figsize=(12, 6),
    edgecolor="black",
    color=[category_color_map[cat] for cat in monthly_category_spending.columns]
)
for i, value in enumerate(monthly_totals):
    plt.text(i, value + 1, f"${value:.2f}", ha="center", va="bottom", fontsize=9)
    
# Add average line
plt.axhline(y=average_spending, color="#117A65", linestyle="dashed", linewidth=2, label=f"Avg: ${average_spending:.2f}")

plt.title(f"Credit Card Monthly Spending by Category 2024 - Total: ${total_spent:,.2f} ")
plt.xlabel("Month")
plt.ylabel("Total Spending ($)")
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Move legend outside plot
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()
###
# monthly_spending = df2024_spending.groupby("Month")["Amount"].sum()
# average_spending = df2024_spending["Amount"].mean()
# total_spent = df2024_spending["Amount"].abs().sum()

# plt.figure(figsize=(10, 5))
# monthly_spending.plot(kind="bar", color="#F5A623", edgecolor="black")
# plt.axhline(y=average_spending, color="#117A65", linestyle="--", linewidth=2, label=f"Avg: ${average_spending:.2f}")

# for i, value in enumerate(monthly_spending):
#     plt.text(i, value + 1, f"${value:.2f}", ha="center", va="bottom", fontsize=9)
# plt.legend()
# plt.title(f"Credit Card Monthly Spending 2024 - Total: ${total_spent:,.2f}")
# plt.xlabel("Month")
# plt.ylabel("Total Credit Card Spending ($)")
# plt.xticks(rotation=45)
# plt.grid(axis="y", linestyle="--", alpha=0.7)
# plt.show()

In [ ]:
# PIE CHART 2024
# spending by category
spending = df2024[df2024["Type"] == 'Sale'].copy()
spending["Amount"] = spending["Amount"].abs()
total_spent = spending["Amount"].sum()

# Group by category (assuming the column is named 'category')
category_spending = spending.groupby("Category")["Amount"].sum().sort_values(ascending=False).reset_index()

fig = px.pie(
    category_spending,
    values="Amount",
    names="Category",
    title=f"Spending Distribution by Category - 2024 (Total: ${total_spent:,.2f})",
    color_discrete_sequence=px.colors.sequential.Blues,
    hole=0.3  # For a donut chart look; remove this line if you want a regular pie
)

fig.update_traces(
    textposition="auto",  # Automatically choose inside or outside
    textinfo="label+percent",
    textfont_size=12,
    insidetextorientation="radial",
    pull=[0.03 if v < 0.05 * total_spent else 0 for v in category_spending["Amount"]],  # Slightly pull out small slices
    showlegend=True
)

# Force small slices' labels to go outside if they won't fit
fig.update_layout(
    uniformtext_minsize=12,
    uniformtext_mode="hide",  # Hide if text can't fit
    margin=dict(t=60, b=40, l=40, r=40),
    showlegend=False,
)


fig.show()

In [ ]:
df_2023= pd.read_csv(os.path.join(os.path.expanduser("~"), "work", "Downloads", "finances", "chase", "Chase1567_Activity2023_Lab.csv"))
df_2023["DATE"] = pd.to_datetime(df_2023["DATE"], errors="coerce")

spending = df_2023[df_2023["AMOUNT"] > 0].copy()
spending["Month"] = spending["DATE"].dt.to_period("M")
monthly_spending = spending.groupby("Month")["AMOUNT"].sum()
average_spending = monthly_spending.mean()
total_spending = spending['AMOUNT'].sum()

plt.figure(figsize=(10, 5))
ax = monthly_spending.plot(kind="bar", color="#FFA500", edgecolor="black")

plt.axhline(y=average_spending, color="#117A65", linestyle="--", linewidth=2, label=f"Avg: ${average_spending:.2f}")
plt.legend()

for i, value in enumerate(monthly_spending):
    plt.text(i, value + 1, f"${value:.2f}", ha="center", va="bottom", fontsize=9)

plt.title(f"Monthly Credit Card Spending 2023 - Total: ${total_spending:.2f}" )
plt.xlabel("Month")
plt.ylabel("Total Credit Card Spending ($)")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# PIE CHART 2023
# spending by category
spending = df_2023[df_2023["AMOUNT"] > 0].copy()
total_spent = spending["AMOUNT"].sum()

# Group by category (assuming the column is named 'category')
category_spending = spending.groupby("CATEGORY")["AMOUNT"].sum().sort_values(ascending=False).reset_index()

fig = px.pie(
    category_spending,
    values="AMOUNT",
    names="CATEGORY",
    title=f"Spending Distribution by Category - 2023 (Total: ${total_spent:,.2f})",
    color_discrete_sequence=px.colors.sequential.Blues,
    hole=0.3  # For a donut chart look; remove this line if you want a regular pie
)

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(
    showlegend=False,
    margin=dict(t=60, b=40, l=40, r=40)
)


fig.show()

In [ ]:
df_2022= pd.read_csv(os.path.join(os.path.expanduser("~"), "work", "Downloads", "finances", "chase", "Chase1567_Activity2022.csv"))
df_2022["DATE"] = pd.to_datetime(df_2022["DATE"], errors="coerce")

spending = df_2022[df_2022["AMOUNT"] > 0].copy()
spending["Month"] = spending["DATE"].dt.to_period("M")
monthly_spending = spending.groupby("Month")["AMOUNT"].sum()
average_spending = monthly_spending.mean()
total_spending = spending['AMOUNT'].sum()

plt.figure(figsize=(10, 5))
ax = monthly_spending.plot(kind="bar", color="#FFA500", edgecolor="black")

plt.axhline(y=average_spending, color="#117A65", linestyle="--", linewidth=2, label=f"Avg: ${average_spending:.2f}")
plt.legend()

for i, value in enumerate(monthly_spending):
    plt.text(i, value + 1, f"${value:.2f}", ha="center", va="bottom", fontsize=9)

plt.title(f"Monthly Credit Card Spending 2022 - Total: ${total_spending:.2f}")
plt.xlabel("Month")
plt.ylabel("Total Credit Card Spending ($)")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# PIE CHART 2022
# spending by category
spending = df_2022[df_2022["AMOUNT"] > 0].copy()
total_spent = spending["AMOUNT"].sum()

# Group by category (assuming the column is named 'category')
category_spending = spending.groupby("CATEGORY")["AMOUNT"].sum().sort_values(ascending=False).reset_index()

fig = px.pie(
    category_spending,
    values="AMOUNT",
    names="CATEGORY",
    title=f"Spending Distribution by Category - 2022 (Total: ${total_spent:,.2f})",
    color_discrete_sequence=px.colors.sequential.Blues,
    hole=0.3  # For a donut chart look; remove this line if you want a regular pie
)

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(
    showlegend=False,
    margin=dict(t=60, b=40, l=40, r=40)
)


fig.show()

In [ ]:
df_2021= pd.read_csv(os.path.join(os.path.expanduser("~"), "work", "Downloads", "finances", "chase", "Chase1567_Activity2021.csv"))
df_2021["DATE"] = pd.to_datetime(df_2021["DATE"], errors="coerce")

spending = df_2021[df_2021["AMOUNT"] > 0].copy()
spending["Month"] = spending["DATE"].dt.to_period("M")
monthly_spending = spending.groupby("Month")["AMOUNT"].sum()
average_spending = monthly_spending.mean()
total_spending = spending['AMOUNT'].sum()

plt.figure(figsize=(10, 5))
ax = monthly_spending.plot(kind="bar", color="#FFA500", edgecolor="black")

plt.axhline(y=average_spending, color="#117A65", linestyle="--", linewidth=2, label=f"Avg: ${average_spending:.2f}")
plt.legend()

for i, value in enumerate(monthly_spending):
    plt.text(i, value + 1, f"${value:.2f}", ha="center", va="bottom", fontsize=9)

plt.title(f"Monthly Credit Card Spending 2022 - Total: ${total_spending:.2f}")
plt.xlabel("Month")
plt.ylabel("Total Credit Card Spending ($)")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# PIE CHART 2021
# spending by category
spending = df_2021[df_2021["AMOUNT"] > 0].copy()
total_spent = spending["AMOUNT"].sum()

# Group by category (assuming the column is named 'category')
category_spending = spending.groupby("CATEGORY")["AMOUNT"].sum().sort_values(ascending=False).reset_index()

fig = px.pie(
    category_spending,
    values="AMOUNT",
    names="CATEGORY",
    title=f"Spending Distribution by Category - 2021 (Total: ${total_spent:,.2f})",
    color_discrete_sequence=px.colors.sequential.Blues,
    hole=0.3  # For a donut chart look; remove this line if you want a regular pie
)

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(
    showlegend=False,
    margin=dict(t=60, b=40, l=40, r=40)
)


fig.show()

## CHARLES SCHWAB WITHDRAWALS

In [ ]:
import calendar
import plotly.graph_objects as go

df_cs= pd.read_csv(os.path.join(os.path.expanduser("~"), "work", "Downloads", "finances", "charles_schwab", "Checking_XXX845_Checking_Transactions_2023_2025.csv"))

# Convert withdrawals to numeric
df_cs['Withdrawal'] = df_cs['Withdrawal'].str.replace(r'[$,]', '', regex=True).astype(float).fillna(0)

# Extract date components
df_cs['Date'] = pd.to_datetime(df_cs['Date'])
df_cs['year'] = df_cs['Date'].dt.year
df_cs['month'] = df_cs['Date'].dt.month

# Aggregate data
monthly_withdrawals = df_cs.groupby(['year', 'month'])['Withdrawal'].sum().reset_index()
annual_avg_monthly = monthly_withdrawals.groupby('year')['Withdrawal'].mean().reset_index()
annual_avg_monthly.rename(columns={'Withdrawal': 'AvgMonthlyWithdrawal'}, inplace=True)

annual_stats = df_cs.groupby('year').agg(
    AvgAmtPerTxn=('Withdrawal', 'mean'),
    TotalWithdrawn=('Withdrawal', 'sum')
).reset_index()

monthly_txns = df_cs.groupby(['year', 'month']).size().reset_index(name='MonthlyTxns')
avg_monthly_txns = monthly_txns.groupby('year')['MonthlyTxns'].mean().reset_index()


# Create faceted bar chart
fig = px.bar(monthly_withdrawals, 
             x='month', 
             y='Withdrawal',
             facet_row='year',
             color_discrete_sequence=['#2CAEA0'],  # Teal color
             labels={'month': 'Month', 'Withdrawal': 'Withdrawal Amount'},
             title='Monthly Withdrawals with Annual Averages',
             height=800,
             width=1000)

annual_totals = annual_stats[['year', 'TotalWithdrawn']]
for annotation in fig.layout.annotations:
    if annotation.text.startswith('year='):
        year = int(annotation.text.split('=')[1])
        total = annual_totals[annual_totals['year'] == year]['TotalWithdrawn'].values[0]
        annotation.text = f"{year} | Total: ${total:,.2f}"
        
# Add horizontal average lines
years = sorted(monthly_withdrawals['year'].unique())
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for i, year in enumerate(years):
    # Get metrics for current year
    year_avg_txn = annual_stats[annual_stats['year'] == year]['AvgAmtPerTxn'].values[0]
    year_avg_monthly = annual_avg_monthly[annual_avg_monthly['year'] == year]['AvgMonthlyWithdrawal'].values[0]
    
    # Add average amount per transaction line (original)
    fig.add_hline(y=year_avg_txn,
                  line_dash='dash', 
                  line_color='#FF6F61',
                  annotation_text=f'Avg Amt/Txn: ${year_avg_txn:,.2f}',
                  annotation_position='top right',
                  row=i+1, 
                  col=1)
    
    # Add average monthly withdrawal line (new)
    fig.add_hline(y=year_avg_monthly,
                  line_dash='dot',
                  line_color='#6A5ACD',
                  annotation_text=f'Avg Monthly: ${year_avg_monthly:,.2f}',
                  annotation_position='bottom right',
                  row=i+1, 
                  col=1)

# Format x-axis with month names
fig.update_xaxes(tickvals=list(range(1,13)), ticktext=month_names)

# Update layout
fig.update_layout(
    showlegend=False,
    plot_bgcolor='white',
    title_x=0.5,
    title_font_size=20,
    margin=dict(l=50, r=50, b=50, t=80),
    height=200 * len(years)  # Adjust height based on number of years
)

fig.show()

## ML learning model to predict category (TF_IDF)

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, ClassLabel
import torch

# Combine and clean labeled data
df_labeled = pd.concat([df2024, df2025])
df_labeled = df_labeled[df_labeled["Category"].notna()]
type_counts = df_labeled["Category"].value_counts()
valid_types = type_counts[type_counts >= 2].index
df_labeled = df_labeled[df_labeled["Category"].isin(valid_types)]
df_labeled["Description"] = df_labeled["Description"].str.lower()
df_unlabeled = df_2023.copy()

# Encode labels
le = LabelEncoder()
df_labeled["label"] = le.fit_transform(df_labeled["Category"])

# Tokenizer and Dataset
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(example):
    return tokenizer(example["Description"], truncation=True, padding="max_length", max_length=64)

dataset = Dataset.from_pandas(df_labeled[["Description", "label"]])
unique_labels = sorted(set(dataset["label"]))
class_label = ClassLabel(num_classes=len(unique_labels), names=sorted(unique_labels))

# Cast the label column to ClassLabel
dataset = dataset.cast_column("label", class_label)
dataset = dataset.train_test_split(test_size=0.2, stratify_by_column="label")
dataset = dataset.map(tokenize, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Model
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(le.classes_)
)

# Training
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

trainer.train()

# Evaluate
preds_output = trainer.predict(dataset["test"])
y_true = dataset["test"]["label"]
y_pred = np.argmax(preds_output.predictions, axis=1)
labels = list(range(len(le.classes_)))
print(classification_report(
    y_true,
    y_pred,
    labels=labels,
    target_names=le.classes_,
    zero_division=0  # avoids warnings if precision/recall are undefined
))

# Predict on new data
df_unlabeled["DESCR"] = df_unlabeled["DESCR"].fillna("").str.lower()
test_encodings = tokenizer(df_unlabeled["DESCR"].tolist(), truncation=True, padding="max_length", max_length=64, return_tensors="pt")

with torch.no_grad():
    outputs = model(**test_encodings)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    confidences, preds = torch.max(probs, dim=1)

# Apply confidence threshold
confidence_threshold = 0.68
final_labels = [
    le.inverse_transform([pred.item()])[0] if conf >= confidence_threshold else np.nan
    for pred, conf in zip(preds, confidences)
]

df_unlabeled["Predicted_Category"] = final_labels
df_unlabeled.to_csv("Chase1567_Activity2022_Labelled_BERT.csv", index=False)
print("✅ BERT-based predictions saved.")